# Random Forests Algorithm Adaptation

In [1]:
import pandas as pd
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn.metrics import hamming_loss
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
import pickle

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
full_data = pd.DataFrame(pd.read_pickle('combined_imdb_tmdb_data.picke'))
full_data.head()

,adult,backdrop_path,belongs_to_collection,budget,cast,certificates,color info,countries,director,genres,homepage,id,imdb_id,languages,mpaa,original_language,original_title,overview,plot,popularity,poster_path,production companies,production_companies,production_countries,rating,release_date,revenue,runtime,runtimes,spoken_languages,status,tagline,title,video,vote_average,vote_count,votes,writer,year
0,False,/nhhV4jSDZcBCZuOjkbPUAGa8EKG.jpg,None,0,"[Alexander Ludwig, Christopher Eccleston, Ian ...","[Argentina:13, Australia:PG, Brazil:Livre, Can...",[Color],[USA],[David L. Cunningham],"[{u'id': 12, u'name': u'Adventure'}, {u'id': 1...",,2274,tt0484562,[English],Rated PG for fantasy action and some scary images,en,The Seeker: The Dark Is Rising,A boy's life is turned upside down when he lea...,"[The film tells the story of Will Stanton, a y...",0.610288,/wmPHQxchlpuRAFG94NWMtCZF65k.jpg,"[Twentieth Century Fox Film Corporation, Walde...",[{u'name': u'Twentieth Century Fox Film Corpor...,"[{u'iso_3166_1': u'US', u'name': u'United Stat...",4.8,2007-10-05,0,94.0,[99],"[{u'iso_639_1': u'en', u'name': u'English'}]",Released,,The Seeker: The Dark Is Rising,False,4.8,88,13504.0,"[John Hodge, Susan Cooper]",2007.0
1,False,/d23XXntJrVwNENvdriJdqlW1ae6.jpg,None,3000000,"[William Holden, Alec Guinness, Jack Hawkins, ...","[Argentina:13, Australia:PG, Australia:G::(Ori...",[Color::(Technicolor)],"[UK, USA]",[David Lean],"[{u'id': 18, u'name': u'Drama'}, {u'id': 36, u...",,826,tt0050212,"[English, Japanese, Thai]",Rated PG for mild war violence (re-rating) (1991),en,The Bridge on the River Kwai,The classic story of English POWs in Burma for...,[The film deals with the situation of British ...,1.138653,/vtPR6tSHeu35rF6qTDw3Yjr9eDg.jpg,"[Columbia Pictures Corporation, Horizon Pictur...","[{u'name': u'Columbia Pictures Corporation', u...","[{u'iso_3166_1': u'GB', u'name': u'United King...",8.2,1957-10-02,33300000,161.0,[161],"[{u'iso_639_1': u'ja', u'name': u'日本語'}, {u'is...",Released,It spans a whole new world of entertainment!,The Bridge on the River Kwai,False,7.6,450,159094.0,"[Pierre Boulle, Carl Foreman, Michael Wilson]",1957.0
2,False,/2Kx2q1GOmYRjxmILEFIusMIGTUc.jpg,"{u'id': 123800, u'poster_path': u'/giaxAC74fht...",2700000,"[Ethan Hawke, Julie Delpy, Vernon Dobtcheff, L...","[Argentina:Atp, Australia:M, Austria:0, Brazil...",[Color],[USA],[Richard Linklater],"[{u'id': 18, u'name': u'Drama'}, {u'id': 10749...",http://www.warnerbros.com/?site=beforesunset#/...,80,tt0381681,"[English, French]",Rated R for language and sexual references (Ap...,en,Before Sunset,Nine years ago two strangers met by chance and...,[Early thirty-something American Jesse Wallace...,1.026352,/cIj6yWJKUjdCCO7vuZQKl0NqCQe.jpg,"[Warner Independent Pictures (WIP), Castle Roc...","[{u'name': u'Castle Rock Entertainment', u'id'...","[{u'iso_3166_1': u'US', u'name': u'United Stat...",8.0,2004-02-10,15992615,80.0,[80],"[{u'iso_639_1': u'en', u'name': u'English'}, {...",Released,What if you had a second chance with the one t...,Before Sunset,False,7.6,580,177647.0,"[Richard Linklater, Julie Delpy, Ethan Hawke, ...",2004.0
3,False,/Dmcv6XpQ9h8AoRIEhD1hayNpd8.jpg,None,0,"[Michael Bolton, Sarah Shahi, Adam Scott, Matt...","[Singapore:NC16, South Korea:18, USA:TV-MA]",[Color],[USA],"[Scott Aukerman, Akiva Schaffer]","[{u'id': 35, u'name': u'Comedy'}, {u'id': 1040...",,430780,tt6343706,[English],NaN,en,"Michael Bolton's Big, Sexy Valentine's Day Spe...",After Santa tells Michael Bolton that he needs...,[After Santa tells Michael Bolton that he need...,0.097190,/fomxGvTf5HBtaupI9Rk4UGg43OB.jpg,"[Comedy Bang! Bang! Productions, Netflix, Pass...","[{u'name': u'Netflix', u'id': 59811}]","[{u'iso_3166_1': u'US', u'name': u'United Stat...",NaN,2017-02-07,0,54.0,[USA:54],"[{u'iso_639_1': u'en', u'name': u'English'}]",Released,,"Michael Bolton's Big, Sexy Valentine's Day Spe...",False,8.0,4,NaN,"[Scott Aukerman, David Ferguson, Mike Hanford,...",2017.0
4,False,/8fYXNzpwjJIUmP6ZtxPO3zp

In [4]:
cols_to_save = ["budget", "adult", "director", "genres", "id", "original_language", "popularity", 
                "rating", "revenue", "runtime", "vote_average", "vote_count", "votes", "writer", "year"]

In [5]:
full_data_condensed = full_data[cols_to_save]

In [6]:
train_test_indices = pd.read_pickle('train_val_test_byid.pkl')

In [7]:
genres = full_data_condensed["genres"]

In [8]:
genre_ids = []
for genre_list in genres:
    genre_ids.append([genre["name"] for genre in genre_list])

In [9]:
full_data_condensed["genres"] = genre_ids

/Users/chrischen/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [10]:
genre_names = []
for genre_sublist in genre_ids:
    genre_names += genre_sublist
genre_names = list(set(genre_names))

In [11]:
genre_names

[u'Mystery',
 u'Romance',
 u'Crime',
 u'Family',
 u'Science Fiction',
 u'Horror',
 u'Foreign',
 u'Drama',
 u'Fantasy',
 u'Western',
 u'Animation',
 u'Music',
 u'Adventure',
 u'Action',
 u'TV Movie',
 u'Comedy',
 u'Documentary',
 u'War',
 u'Thriller',
 u'History']

In [12]:
for genre in genre_names:
    full_data_condensed[genre] = [int(genre in genre_list) for genre_list in full_data_condensed["genres"]]

/Users/chrischen/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [13]:
import numpy as np

In [14]:
np.sum(full_data_condensed["Mystery"])

1172

In [15]:
full_data_condensed.head()

,budget,adult,director,genres,id,original_language,popularity,rating,revenue,runtime,vote_average,vote_count,votes,writer,year,Mystery,Romance,Crime,Family,Science Fiction,Horror,Foreign,Drama,Fantasy,Western,Animation,Music,Adventure,Action,TV Movie,Comedy,Documentary,War,Thriller,History
0,0,False,[David L. Cunningham],"[Adventure, Drama, Fantasy, Family, Thriller]",2274,en,0.610288,4.8,0,94.0,4.8,88,13504.0,"[John Hodge, Susan Cooper]",2007.0,0,0,0,1,0,0,0,1,1,0,0,0,1,0,0,0,0,0,1,0
1,3000000,False,[David Lean],"[Drama, History, War]",826,en,1.138653,8.2,33300000,161.0,7.6,450,159094.0,"[Pierre Boulle, Carl Foreman, Michael Wilson]",1957.0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1
2,2700000,False,[Richard Linklater],"[Drama, Romance]",80,en,1.026352,8.0,15992615,80.0,7.6,580,177647.0,"[Richard Linklater, Julie Delpy, Ethan Hawke, ...",2004.0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3,0,False,"[Scott Aukerman, Akiva Schaffer]","[Comedy, Music]",430780,en,0.097190,NaN,0,54.0,8.0,4,NaN,"[Scott Aukerman, David Ferguson, Mike Hanford,...",2017.0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
4,15000000,False,[Mark Palansky],"[Fantasy, Comedy, Romance]",7985,en,1.222510,6.8,0,103.0,6.5,317,52587.0,[Leslie Caveny],2006.0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0


In [16]:
first_writers = []
for writer_list in full_data_condensed["writer"]:
    if type(writer_list) == list:
        first_writers.append(writer_list[0])
    else:
        first_writers.append(np.nan)

In [17]:
len(list(set(first_writers)))

7486

In [18]:
for column in ["director", "genres", "writer"]:
    full_data_condensed.drop(column, axis=1, inplace=True)

/Users/chrischen/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [19]:
full_data_condensed.head()

,budget,adult,id,original_language,popularity,rating,revenue,runtime,vote_average,vote_count,votes,year,Mystery,Romance,Crime,Family,Science Fiction,Horror,Foreign,Drama,Fantasy,Western,Animation,Music,Adventure,Action,TV Movie,Comedy,Documentary,War,Thriller,History
0,0,False,2274,en,0.610288,4.8,0,94.0,4.8,88,13504.0,2007.0,0,0,0,1,0,0,0,1,1,0,0,0,1,0,0,0,0,0,1,0
1,3000000,False,826,en,1.138653,8.2,33300000,161.0,7.6,450,159094.0,1957.0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1
2,2700000,False,80,en,1.026352,8.0,15992615,80.0,7.6,580,177647.0,2004.0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3,0,False,430780,en,0.097190,NaN,0,54.0,8.0,4,NaN,2017.0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
4,15000000,False,7985,en,1.222510,6.8,0,103.0,6.5,317,52587.0,2006.0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0


In [20]:
full_data_condensed.drop("original_language", axis=1, inplace=True)

/Users/chrischen/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [21]:
full_data_condensed["adult"] = [int(adult) for adult in full_data_condensed["adult"]]

/Users/chrischen/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [22]:
full_data_condensed.head()

,budget,adult,id,popularity,rating,revenue,runtime,vote_average,vote_count,votes,year,Mystery,Romance,Crime,Family,Science Fiction,Horror,Foreign,Drama,Fantasy,Western,Animation,Music,Adventure,Action,TV Movie,Comedy,Documentary,War,Thriller,History
0,0,0,2274,0.610288,4.8,0,94.0,4.8,88,13504.0,2007.0,0,0,0,1,0,0,0,1,1,0,0,0,1,0,0,0,0,0,1,0
1,3000000,0,826,1.138653,8.2,33300000,161.0,7.6,450,159094.0,1957.0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1
2,2700000,0,80,1.026352,8.0,15992615,80.0,7.6,580,177647.0,2004.0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,430780,0.097190,NaN,0,54.0,8.0,4,NaN,2017.0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
4,15000000,0,7985,1.222510,6.8,0,103.0,6.5,317,52587.0,2006.0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0


In [23]:
full_data_condensed.drop("adult", axis=1, inplace=True)

/Users/chrischen/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [25]:
train_rows = [tmdb_id in train_test_indices["train"] for tmdb_id in full_data_condensed["id"]]
val_rows = [tmdb_id in train_test_indices["validation"] for tmdb_id in full_data_condensed["id"]]

In [26]:
df_test = full_data_condensed[[tmdb_id in train_test_indices["test"] for tmdb_id in full_data_condensed["id"]]]

In [27]:
len(df_test["id"]) - len(train_test_indices["test"])

-17

In [28]:
with open("test_condensed.pickle", "wb") as handle:
    pickle.dump(df_test, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [29]:
df_train = full_data_condensed[train_rows]
df_val = full_data_condensed[val_rows]

In [31]:
len(set(train_test_indices["validation"]) - set(list(df_val["id"])))

21

In [ ]:
df_train.to_csv("full_data_train.csv")
df_test.to_csv("full_data_test.csv")
df_val.to_csv("full_data_val.csv")

In [ ]:
train_test_indices.keys()

In [ ]:
imdb5k = pd.read_pickle('imdb5000onehot.pkl')
imdb5k.head()

In [ ]:
imdb5k.columns

In [ ]:
# print categoricals
for col in imdb5k.columns:
    if imdb5k[col].dtype == 'object':
        print col

In [ ]:
imdb5k['plot_keywords'].head()

In [ ]:
# one-hot encode select categoricals
# good_categories = ['color', 'country', 'content_rating']
good_categories = []
imdb_quant = pd.DataFrame()
for col in imdb5k.columns:
    if imdb5k[col].dtype != 'object':
        imdb_quant[col] = imdb5k[col]
    elif col in good_categories:
        imdb_quant = pd.concat([imdb_quant, pd.get_dummies(imdb5k[col])], axis=1)

In [ ]:
# x y split
imdb5k_X = imdb_quant.ix[:,0:list(imdb_quant.columns).index('i_Sci-Fi')]
imdb5k_Y = imdb_quant.ix[:,list(imdb_quant.columns).index('i_Sci-Fi'):len(imdb_quant.columns)]

# impute
def impute(dataf):
    for col in dataf.columns:
        dataf[col].fillna(dataf[col].mean(), inplace=True)
impute(imdb5k_X)
impute(imdb5k_Y)

# train test split
x_train, x_test, y_train, y_test = train_test_split(imdb5k_X, imdb5k_Y, stratify = imdb5k_Y)

In [ ]:
params = {
    'n_estimators': [100],
    'max_features': ['auto', 'log2', None, 1] + range(5, 20, 5),
    'n_jobs' : [-1]
}

CV_rf = GridSearchCV(RandomForestClassifier(), param_grid=params, cv=5, n_jobs=-1)
CV_rf.fit(x_train, y_train)
print CV_rf.best_params_
print CV_rf.best_score_
print CV_rf.score(x_test, y_test)

In [ ]:
hamming_loss(y_test, CV_rf.predict(x_test))

In [ ]:
pd.DataFrame({'feature': x_train.columns, 'importance': CV_rf.best_estimator_.feature_importances_})

In [ ]:
# x y split
imdb5k_X = imdb_quant.ix[:,0:list(imdb_quant.columns).index('i_Sci-Fi')]
imdb5k_Y = imdb_quant.ix[:,list(imdb_quant.columns).index('i_Sci-Fi'):len(imdb_quant.columns)]

# impute
def impute(dataf):
    for col in dataf.columns:
        dataf[col].fillna(dataf[col].mean(), inplace=True)
impute(imdb5k_X)
impute(imdb5k_Y)

# train test split
x_train, x_test, y_train, y_test = train_test_split(imdb5k_X, imdb5k_Y, stratify = imdb5k_Y)

## On TMDb + IMDb Data

In [ ]:
df_train.drop("id", axis=1, inplace=True)
df_test.drop("id", axis=1, inplace=True)

# x y split
train_X = df_train.ix[:,0:list(df_train.columns).index('Mystery')]
train_Y = df_train.ix[:,list(df_train.columns).index('Mystery'):len(df_train.columns)]

test_X = df_test.ix[:,0:list(df_test.columns).index('Mystery')]
test_Y = df_test.ix[:,list(df_test.columns).index('Mystery'):len(df_test.columns)]

# impute
def impute(dataf):
    for col in dataf.columns:
        dataf[col].fillna(dataf[col].mean(), inplace=True)

impute(train_X)
impute(train_Y)
impute(test_X)
impute(test_Y)

In [ ]:
train_X.columns

In [ ]:
params = {
    'n_estimators': [100],
    'max_features': ['auto', 'log2', None, 1, 3, 5, 7, 9],
    'n_jobs' : [-1]
}

CV_rf = GridSearchCV(RandomForestClassifier(), param_grid=params, cv=5, n_jobs=-1)
CV_rf.fit(train_X, train_Y)
print CV_rf.best_params_
print CV_rf.best_score_
print CV_rf.score(test_X, test_Y)

In [ ]:
hamming_loss(test_Y, CV_rf.predict(test_X))

In [ ]:
pd.DataFrame({'feature': train_X.columns, 'importance': CV_rf.best_estimator_.feature_importances_})